In [1]:
import json
import os
from pandas import DataFrame as df
import pandas as pd


# read files

In [2]:
IMPORT_PATH = "../data_enflate_cleaned"
filenames = os.listdir(IMPORT_PATH)
filenames = [file for file in filenames if file.endswith(".json")]
filename_list = [IMPORT_PATH + "/" + filename for filename in filenames]
devices_data = {}
for file in filename_list:
    device_data = pd.read_json(file)
    #print(device_data.head())
    devices_data[file] = device_data

In [3]:
devices_data_working = {name:d.copy() for name, d in devices_data.items()}
devices_data_small = devices_data.copy()
#print(devices_data_working.keys())
print(f"found {len(devices_data_working.keys())} devices")

date_limit = pd.Timestamp("2024-08-30")

allowed_headers = ["Device", "Timestamp", "L1_min_current", "L1_max_current", "relay_state", "L1_active_energy_diff"]
for name, device in devices_data_working.items():
    headers = device.columns
    #print(headers)

    # delete all colums not in headers
    for column in device.columns:
        if column not in allowed_headers:
            del device[column]
    
    # delete all rows with timestamp <  date_limit
    device = device[device["Timestamp"] > date_limit]
    print(name)
    print(device.head())


found 4 devices
../data_enflate_cleaned/site_6037.json
       Device           Timestamp  L1_max_current  L1_min_current  \
12503  Boiler 2024-08-30 00:00:29              76              48   
12504  Boiler 2024-08-30 00:00:59              99              48   
12505  Boiler 2024-08-30 00:01:29              90              48   
12506  Boiler 2024-08-30 00:01:59              85              48   
12507  Boiler 2024-08-30 00:02:29              74              51   

       relay_state  L1_active_energy_diff  
12503            1                      0  
12504            1                      0  
12505            1                      0  
12506            1                      0  
12507            1                      0  
../data_enflate_cleaned/site_6128.json
       Device           Timestamp  L1_max_current  L1_min_current  \
51946  Boiler 2024-08-30 07:19:03              78              42   
51947  Boiler 2024-08-30 07:25:03             115              42   
51948  Boiler 2024-0

# Calculate KPIs

## Anzahl Schaltungen

In [4]:
def calculate_switching_count(device_data) -> int:
    """loop over df and count the number of times the L1_min_voltage differs from L1_max_voltage"""
    switching_count = 0
    for index, row in device_data.iterrows():
        # diff smaler 1 ampere
        if abs(row["L1_min_current"] - row["L1_max_current"]) > 50:
            switching_count += 1

    # TODO: pmean per day
        
    return switching_count

## respects Rundsteuerung
Device stays turned of during the full period

In [5]:
def respects_signal(device_data, diff_limit) -> bool:
    """check if the device respects the signal relay_state"""

    # plot L1_active_energy over time, limit max valuer to 50
    # device_data.plot(x="Timestamp", y="L1_active_energy_diff", ylim=(0, 5000))

    conflict_count = 0
    correct_count = 0
    respects_signal = True
    for index, row in device_data.iterrows():
        if row["relay_state"] == 0 and row["L1_active_energy_diff"] > diff_limit:
            conflict_count += 1
        if row["relay_state"] == 0 and row["L1_active_energy_diff"] < diff_limit:
            correct_count += 1
    print(f"conflict_count: {conflict_count} correct_count: {correct_count} ratio: {conflict_count*100/correct_count}%")
    # tolerating 1% of conflicting
    if conflict_count/correct_count > 0.01:
        respects_signal = False
    return respects_signal



## react to Rundsteuernung

In [12]:
def reacts_to_signal(device_data, device_name):# -> react_count_on_block_count, react_count_on_release_count:
    """check if the device reacts to a signal"""

    SWITCH_DIFFERENCE = 50

    signal_point_0 = 0
    signal_point_1 = 0

    react_count_on_block_count = 0
    react_count_on_release_count = 0

    for index, row in device_data.iterrows():

        # start not from beginning
        if index > 5:
            signal_point_1 = row["relay_state"]

            # if row["Timestamp"].date() == pd.Timestamp("2024-08-31").date():
                # print(f"device_name {device_name}  {row['Timestamp']} {row['relay_state']} {row['L1_active_energy_diff']}")
            # from now consumption blocked
            if signal_point_0 > signal_point_1:
                # if date is 31.8.2024 00:00:00

                L1_active_energy_diff_minus_2 = device_data.iloc[index - 2]["L1_active_energy_diff"]
                L1_active_energy_diff_plus_2 = device_data.iloc[index + 2]["L1_active_energy_diff"]

                # detect cunsumption switch
                if ( L1_active_energy_diff_minus_2 - L1_active_energy_diff_plus_2 ) > SWITCH_DIFFERENCE:
                    react_count_on_block_count += 1
                    # print(f"device_name {device_name}  {device_data.iloc[index - 2]['Timestamp']}   {device_data.iloc[index - 2]['relay_state']}   {device_data.iloc[index - 2]['L1_active_energy_diff']}")
                    # print(f"device_name {device_name}  {device_data.iloc[index - 1]['Timestamp']}   {device_data.iloc[index - 1]['relay_state']}   {device_data.iloc[index - 1]['L1_active_energy_diff']}")
                    # print(f"device_name {device_name}  {device_data.iloc[index - 0]['Timestamp']}   {device_data.iloc[index - 0]['relay_state']}   {device_data.iloc[index - 0]['L1_active_energy_diff']}")
                    # print(f"device_name {device_name}  {device_data.iloc[index + 1]['Timestamp']}   {device_data.iloc[index + 1]['relay_state']}   {device_data.iloc[index + 1]['L1_active_energy_diff']}")
                    # print(f"device_name {device_name}  {device_data.iloc[index + 1]['Timestamp']}   {device_data.iloc[index + 1]['relay_state']}   {device_data.iloc[index + 2]['L1_active_energy_diff']}")
                # else:
                #     react_count_on_release_count += 1
            
            # from now cunsomption allowed
            elif signal_point_0 < signal_point_1:
                L1_active_energy_diff_minus_2 = device_data.iloc[index - 2]["L1_active_energy_diff"]
                L1_active_energy_diff_plus_2 = device_data.iloc[index - 2]["L1_active_energy_diff"]

                # detect cunsumption switch
                if ( L1_active_energy_diff_plus_2 - L1_active_energy_diff_plus_2 ) > SWITCH_DIFFERENCE:
                    react_count_on_release_count += 1
                    # print(f"device_name {device_name}  {device_data.iloc[index - 2]['Timestamp']}   {device_data.iloc[index - 2]['relay_state']}   {device_data.iloc[index - 2]['L1_active_energy_diff']}")
                    # print(f"device_name {device_name}  {device_data.iloc[index - 1]['Timestamp']}   {device_data.iloc[index - 1]['relay_state']}   {device_data.iloc[index - 1]['L1_active_energy_diff']}")
                    # print(f"device_name {device_name}  {device_data.iloc[index - 0]['Timestamp']}   {device_data.iloc[index - 0]['relay_state']}   {device_data.iloc[index - 0]['L1_active_energy_diff']}")
                    # print(f"device_name {device_name}  {device_data.iloc[index + 1]['Timestamp']}   {device_data.iloc[index + 1]['relay_state']}   {device_data.iloc[index + 1]['L1_active_energy_diff']}")
                    # print(f"device_name {device_name}  {device_data.iloc[index + 1]['Timestamp']}   {device_data.iloc[index + 1]['relay_state']}   {device_data.iloc[index + 2]['L1_active_energy_diff']}")

                # else:
                #     react_count_on_release_count += 1

        # rollover values
        signal_point_0 = signal_point_1
    
    return react_count_on_block_count, react_count_on_release_count
    # # make decision
    # if react_count_on_block_count > 0:
    #     print(f"true react_count: {react_count} react_coureact_count_on_block_countnt_on_release_count: {react_count_on_release_count} ratio: {react_count*100/react_count_on_release_count}%")
    #     return True
    # else:
    #     print(f"false react_count: {react_count} react_count_on_release_count: {react_count_on_release_count} ratio: {react_count*100/react_count_on_release_count}%")
    #     return False
    

In [13]:
devices_kpis = {}
device_type_kpis = {}
diff_limit = 50

for device_name, device_data in devices_data_working.items():
    device_kpis = {}
    #device_kpis["switching_count"] = calculate_switching_count(device_data)
    #device_kpis["respects_signal"] = respects_signal(device_data, diff_limit)
    #if device_name == '../data_enflate_cleaned/site_6128.json':

    # Rundsteuerung reaction
    react_count_on_block_count, react_count_on_release_count = reacts_to_signal(device_data, device_name)
    device_kpis["react_count_on_block_count"] = react_count_on_block_count
    device_kpis["react_count_on_release_count"] = react_count_on_release_count

    # Device type
    device_kpis["Device"] = device_data["Device"].iloc[0]


    short_device_name = device_name.split("/")[-1].split(".")[0]
    # Print KPIs
    devices_kpis[short_device_name] = device_kpis
    print (f" {short_device_name} PKIs: {device_kpis}")

    if device_kpis["Device"] not in device_type_kpis:
        device_type_kpis[device_kpis["Device"]] = {}
    if "react_count_on_block_count_values" not in device_type_kpis[device_kpis["Device"]]:
        device_type_kpis[device_kpis["Device"]]["react_count_on_block_count_values"] = []
    device_type_kpis[device_kpis["Device"]]["react_count_on_block_count_values"].append(device_kpis["react_count_on_block_count"])
    if "react_count_on_release_count_values" not in device_type_kpis[device_kpis["Device"]]:
        device_type_kpis[device_kpis["Device"]]["react_count_on_release_count_values"] = []
    device_type_kpis[device_kpis["Device"]]["react_count_on_release_count_values"].append(device_kpis["react_count_on_release_count"])
    
    # plot L1_min_voltage and L1_max_voltage
    # device_data.plot(x='Timestamp', y=['L1_min_current', 'L1_max_current'])


    

 site_6037 PKIs: {'react_count_on_block_count': 2, 'react_count_on_release_count': 0, 'Device': 'Boiler'}
 site_6128 PKIs: {'react_count_on_block_count': 1, 'react_count_on_release_count': 0, 'Device': 'Boiler'}
 site_7915 PKIs: {'react_count_on_block_count': 1, 'react_count_on_release_count': 0, 'Device': 'Boiler'}
 site_7966 PKIs: {'react_count_on_block_count': 1, 'react_count_on_release_count': 0, 'Device': 'Boiler'}


# Print statistics


In [14]:

print("Per device type KPIs")
print("----------------")
print ( json.dumps( device_type_kpis, indent=4, sort_keys=True) )

print("")
print("Per device KPIs")
print("----------------")
print ( json.dumps( devices_kpis, indent=4, sort_keys=True) )



Per device KPIs
----------------
{
    "site_6037": {
        "Device": "Boiler",
        "react_count_on_block_count": 2,
        "react_count_on_release_count": 0
    },
    "site_6128": {
        "Device": "Boiler",
        "react_count_on_block_count": 1,
        "react_count_on_release_count": 0
    },
    "site_7915": {
        "Device": "Boiler",
        "react_count_on_block_count": 1,
        "react_count_on_release_count": 0
    },
    "site_7966": {
        "Device": "Boiler",
        "react_count_on_block_count": 1,
        "react_count_on_release_count": 0
    }
}

Per device type KPIs
----------------
{
    "Boiler": {
        "react_count_on_block_count_values": [
            2,
            1,
            1,
            1
        ],
        "react_count_on_release_count_values": [
            0,
            0,
            0,
            0
        ]
    }
}
